# Loan Prediction

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
from numpy import mean
from numpy import std
from numpy import absolute

In [2]:
df_train = pd.read_csv('/Users/tsaichichien/BADM Data Science /Loan_Prediction/Data for Cleaning & Modeling.csv')
df_test = pd.read_csv('/Users/tsaichichien/BADM Data Science /Loan_Prediction/Holdout for Testing.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_train.columns

Index(['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11',
       'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21',
       'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31',
       'X32'],
      dtype='object')

# Data Cleaning

In [4]:
# Check Training data info
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 32 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   X1      338990 non-null  object 
 1   X2      399999 non-null  float64
 2   X3      399999 non-null  float64
 3   X4      399999 non-null  object 
 4   X5      399999 non-null  object 
 5   X6      399999 non-null  object 
 6   X7      399999 non-null  object 
 7   X8      338730 non-null  object 
 8   X9      338730 non-null  object 
 9   X10     376014 non-null  object 
 10  X11     382462 non-null  object 
 11  X12     338639 non-null  object 
 12  X13     338972 non-null  float64
 13  X14     399999 non-null  object 
 14  X15     399999 non-null  object 
 15  X16     123560 non-null  object 
 16  X17     399999 non-null  object 
 17  X18     399981 non-null  object 
 18  X19     399999 non-null  object 
 19  X20     399999 non-null  object 
 20  X21     399999 non-null  float64
 21  X22     39

In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 32 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X1      0 non-null      float64
 1   X2      80000 non-null  int64  
 2   X3      80000 non-null  int64  
 3   X4      80000 non-null  object 
 4   X5      80000 non-null  object 
 5   X6      80000 non-null  object 
 6   X7      80000 non-null  object 
 7   X8      80000 non-null  object 
 8   X9      80000 non-null  object 
 9   X10     75606 non-null  object 
 10  X11     75618 non-null  object 
 11  X12     80000 non-null  object 
 12  X13     80000 non-null  float64
 13  X14     80000 non-null  object 
 14  X15     80000 non-null  object 
 15  X16     15 non-null     object 
 16  X17     80000 non-null  object 
 17  X18     80000 non-null  object 
 18  X19     80000 non-null  object 
 19  X20     80000 non-null  object 
 20  X21     80000 non-null  float64
 21  X22     80000 non-null  int64  
 22

In [6]:
df_train.shape

(400000, 32)

In [7]:
#Missing Value check
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
X26,348845,0.872112
X16,276440,0.691100
X25,218802,0.547005
X12,61361,0.153402
X8,61270,0.153175
...,...,...
X29,1,0.000003
X31,1,0.000003
X24,1,0.000003
X23,1,0.000003


## Fill NA column by column:

Drop X2 X3 as identifier

In [8]:
# drop X2 and X3 from training and test set
df_train.drop(['X2', 'X3'], axis=1, inplace=True)
df_test.drop(['X2', 'X3'], axis=1, inplace=True)

1. X1 (depedent variable) got a missing values of 61010 records

In [9]:
# remove '%' from interest rate column and change dtype to numeric
df_train['X1'] = pd.to_numeric(df_train['X1'].str.replace('%', ''))/100

# drop the rows with missing interest rate values
df_train = df_train.dropna(subset=['X1'])

2. Remove X4: 
One row within the training data set with all missing values besides the loan interest rate value. Here we will remove this row:

In [10]:
df_train.dropna(subset=['X4'], inplace=True)

3. Remove X8, X9 column, since it got a mnissing values over 61,000 records

In [11]:
# drop X8 and X9 columns: loan grade and subgrade
df_train.dropna(subset=['X8', 'X9'], inplace=True)

4. Remove column X16: Description is not useful for Prediction analysis

In [12]:
# remove X16 from training set
df_train.drop(['X16'], axis=1, inplace=True)

# remove X16 from test set
df_test.drop(['X16'], axis=1, inplace=True)

5. Remove column X10: Borrower-provided job title
Unique job title of customers is not useful for prediction

In [13]:
# remove X10 from training set
df_train.drop(['X10'], axis=1, inplace=True)

# remove X10 variable from tests set
df_test.drop(['X10'], axis=1, inplace=True)

6. Remove rows where X13 column has missing values: Annual Income
Although there are more than 61000 missing values, Annual Income can be a good indicator for prediction, thus, we only remove the missing values

In [14]:
# drop rows where there are missing values for X13 column from data set
df_train.dropna(subset=['X13'], inplace=True)

7. Remove X18 column: Loan Title
The loan title is only provided by the borrowers, thus, the name is not united. 

In [15]:
# remove X18 from training set
df_train.drop(['X18'], axis=1, inplace=True)

# remove X18 variable from test set
df_test.drop(['X18'], axis=1, inplace=True)

8. Remove X25 and X26 from data sets: # of months since last delinquency and public record
Both of the columns got a more than 50% of missing values, thus, we can just remove both columns.

In [16]:
# remove X25, X26 from training set
df_train.drop(['X25', 'X26'], axis=1, inplace=True)

# remove X25, X26 variable from test set
df_test.drop(['X25', 'X26'], axis=1, inplace=True)

9. Remove rows with missing values in X30 column: credit utilization percentage
Since there sre only 154 missing values, we will delete from both the training and the test sets

In [17]:
# drop rows where there are missing values for X30 column from data set
df_train.dropna(subset=['X30'], inplace=True)

# convert X30 to numeric data type
df_test['X30'] = pd.to_numeric(df_test['X30'].str.replace('%', ''))

# fill rows in test data where there are missing values for X30 column with median values
df_test['X30'] = df_test['X30'].fillna(df_test.X30.median(axis=0))

10. Remove X17: Loan Category (provided by Borrower) from data sets...
Category is not important for prediction of interest rate

In [18]:
# remove X10 from training set
df_train.drop(['X17'], axis=1, inplace=True)

# remove X10 variable from test set
df_test.drop(['X17'], axis=1, inplace=True)

11. Remove X1 empty column from test set 

In [19]:
df_test.drop(['X1'], axis=1, inplace=True)

11. Remove X11 (Years of Employment) column from training and test sets
Both dataset got 'n/a' values, however, we don't want to drop the n/a value columns. Therefore, at this point, we will remove the column for both data set

In [20]:
df_test.drop(['X11'], axis=1, inplace=True)
df_train.drop(['X11'], axis=1, inplace=True)

12. Remove X15(Date Loan was Issued) & X23(Date of Earliest Line of Credit)
Both values has no meaning for loan interest rate prediction

In [21]:
# Remove X15
df_test.drop(['X15'], axis=1, inplace=True)
df_train.drop(['X15'], axis=1, inplace=True)
#Remove
df_test.drop(['X23'], axis=1, inplace=True)
df_train.drop(['X23'], axis=1, inplace=True)

13. Remove X20 column (State)
Since we already have X19 column for area code, we can remove X20 state name

In [22]:
# drop X20 from trainig and test sets
df_test.drop(['X20'], axis=1, inplace=True)
df_train.drop(['X20'], axis=1, inplace=True)

14. Remove X32 column (Initial Listing Status of Loan) from training and test set
Initial Listing status of loan will not have any effect on the Loan interest rate

In [23]:
df_test.drop(['X32'], axis=1, inplace=True)
df_train.drop(['X32'], axis=1, inplace=True)

15. Remove X12 Column (Home Ownership Status)
Since there are a few ambiguous categories (i.e. NONE, OTHER, & ANY) AND nan values. It is impossible to have a accurate prediction result.

In [24]:
print("Unique categories in Home Ownership variable: ", df_train['X12'].unique())

Unique categories in Home Ownership variable:  ['RENT' 'OWN' 'MORTGAGE' 'NONE' nan 'OTHER' 'ANY']


In [25]:
df_test.drop(['X12'], axis=1, inplace=True)
df_train.drop(['X12'], axis=1, inplace=True)

In [26]:
# Convert string columns to appropriate numeric data types in training data
# Covert X4-X7 from string to numeric by using pd.to_numeric 
# Remove Dollar sign by using str.replace function 
df_train['X4'] = df_train['X4'].str.replace('$', '')
df_train['X4'] = pd.to_numeric(df_train['X4'].str.replace(',', ''))
df_train['X5'] = df_train['X5'].str.replace('$', '')
df_train['X5'] = pd.to_numeric(df_train['X5'].str.replace(',', ''))
df_train['X6'] = df_train['X6'].str.replace('$', '')
df_train['X6'] = pd.to_numeric(df_train['X6'].str.replace(',', ''))
df_train['X7'] = pd.to_numeric(df_train['X7'].str.replace(' months', ''))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


In [27]:
# Convert string to interger type by using astype(int)
# Replace x sign from X19
df_train['X19'] = pd.to_numeric(df_train['X19'].str.replace('x', ''))
df_train['X21'] = df_train['X21']/100 
df_train['X13'] = df_train['X13'].astype(int) 
df_train['X22'] = df_train['X22'].astype(int)
df_train['X24'] = df_train['X24'].astype(int) 
df_train['X27'] = df_train['X27'].astype(int) 
df_train['X28'] = df_train['X28'].astype(int) 
df_train['X29'] = df_train['X29'].astype(int) 
df_train['X30'] = df_train['X30'].str.replace('%', '')
df_train['X30'] = pd.to_numeric(df_train['X30'].str.replace(',', ''))/100
df_train['X31'] = df_train['X31'].astype(int) 

In [28]:
# Encoding to numeric data (X8,X14)
code_numeric = {'not verified': 1, 'VERIFIED - income': 2, 'VERIFIED - income source': 3}

df_train = df_train.applymap(lambda s: code_numeric.get(s) if s in code_numeric else s)
df_test = df_test.applymap(lambda s: code_numeric.get(s) if s in code_numeric else s)


In [29]:
# Encode X9
# Create integer labels for categorical variables in training data
# instatiate sklearn's labelencoder
le = preprocessing.LabelEncoder()

# fit label encorder and return encoded integer labels for categorical string features
df_train['X9'] = le.fit_transform(df_train['X9'].values)
df_train['X8'] = le.fit_transform(df_train['X8'].values)
df_test['X9'] = le.fit_transform(df_test['X9'].values)
df_test['X8'] = le.fit_transform(df_test['X8'].values)


### Final Check on Data type of each column from train and test dataset

In [30]:
df_train.info()
df_train.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242983 entries, 0 to 399999
Data columns (total 18 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   X1      242983 non-null  float64
 1   X4      242983 non-null  int64  
 2   X5      242983 non-null  int64  
 3   X6      242983 non-null  int64  
 4   X7      242983 non-null  int64  
 5   X8      242983 non-null  int64  
 6   X9      242983 non-null  int64  
 7   X13     242983 non-null  int64  
 8   X14     242983 non-null  int64  
 9   X19     242983 non-null  int64  
 10  X21     242983 non-null  float64
 11  X22     242983 non-null  int64  
 12  X24     242983 non-null  int64  
 13  X27     242983 non-null  int64  
 14  X28     242983 non-null  int64  
 15  X29     242983 non-null  int64  
 16  X30     242983 non-null  float64
 17  X31     242983 non-null  int64  
dtypes: float64(3), int64(15)
memory usage: 35.2 MB


,X1,X4,X5,X6,X7,X8,X9,X13,X14,X19,X21,X22,X24,X27,X28,X29,X30,X31
0,0.1189,25000,25000,19080,36,1,8,85000,2,941,0.1948,0,0,10,0,28854,0.521,42
1,0.1071,7000,7000,673,36,1,9,65000,1,112,0.1429,0,0,7,0,33623,0.767,7
2,0.1699,25000,25000,24725,36,3,17,70000,2,100,0.1050,0,0,10,0,19878,0.663,17
3,0.1311,1200,1200,1200,36,2,11,54000,1,777,0.0547,0,0,5,0,2584,0.404,31
4,0.1357,10800,10800,10692,36,2,12,32000,1,67,0.1163,0,1,14,0,3511,0.256,40


In [31]:
df_train.describe()

,X1,X4,X5,X6,X7,X8,X9,X13,X14,X19,X21,X22,X24,X27,X28,X29,X30,X31
count,242983.000000,242983.000000,242983.000000,242983.000000,242983.0000,242983.000000,242983.000000,2.429830e+05,242983.000000,242983.000000,242983.000000,242983.000000,242983.000000,242983.000000,242983.000000,2.429830e+05,242983.000000,242983.000000
mean,0.139406,14267.531062,14237.459102,14163.732150,42.4365,1.812530,11.041484,7.312063e+04,1.986308,514.726899,0.170043,0.275233,0.818831,11.120967,0.153064,1.594181e+04,0.562925,24.979838
std,0.043811,8260.262613,8246.054200,8270.265388,10.6324,1.320213,6.531771,5.657148e+04,0.790897,316.807345,0.076893,0.784393,1.060841,4.887522,0.519584,1.923258e+04,0.237849,11.468621
min,0.054200,500.000000,500.000000,0.000000,36.0000,0.000000,0.000000,3.000000e+03,1.000000,7.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000e+00,0.000000,2.000000
25%,0.109900,8000.000000,8000.000000,8000.000000,36.0000,1.000000,6.000000,4.500000e+04,1.000000,223.000000,0.112700,0.000000,0.000000,8.000000,0.000000,6.465000e+03,0.394000,17.000000
50%,0.136800,12000.000000,12000.000000,12000.000000,36.0000,2.000000,10.000000,6.274400e+04,2.000000,480.000000,0.167100,0.000000,0.000000,10.000000,0.000000,1.177400e+04,0.579000,23.000000
75%,0.167800,20000.000000,20000.000000,19875.000000,60.0000,3.000000,15.000000,8.800000e+04,3.000000,826.000000,0.225000,0.000000,1.000000,14.000000,0.000000,2.022100e+04,0.749000,32.000000
max,0.260600,35000.000000,35000.000000,35000.000000,60.0000,6.000000,34.000000,7.500000e+06,3.000000,999.000000,0.399900,29.000000,8.000000,62.000000,63.000000,2.568995e+06,8.923000,118.000000


In [32]:
# Convert test set data from string to numeric
df_test['X4'] = df_test['X4'].str.replace('$', '')
df_test['X4'] = pd.to_numeric(df_test['X4'].str.replace(',', ''))
df_test['X5'] = df_test['X5'].str.replace('$', '')
df_test['X5'] = pd.to_numeric(df_test['X5'].str.replace(',', ''))
df_test['X6'] = df_test['X6'].str.replace('$', '')
df_test['X6'] = pd.to_numeric(df_test['X6'].str.replace(',', ''))
df_test['X7'] = pd.to_numeric(df_test['X7'].str.replace(' months', ''))
df_test['X19'] = pd.to_numeric(df_test['X19'].str.replace('x', ''))
df_test['X21'] = df_test['X21']/100 # convert dti percentage to ratio
df_test['X13'] = df_test['X13'].astype(int) # convert to interger type
df_test['X30'] = df_test['X30']/100 # convert revolving utilization to ratio

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


# Machine Learning

In [33]:
# Seperate x, y predictor
y = df_train['X1']
X = df_train.drop(['X1'],axis=1)

In [35]:
# Linear Regression Model 
model = LinearRegression()
model.fit(X, y)
# use trained Linear regression model to predict interest rates of training and test data
train_pred = model.predict(X)

LR_test_pred = model.predict(df_test)

# LR_test_pred = absolute(scores)
# print RMSE of training predictions
print('RMSE on training data: ', np.sqrt(mean_squared_error(y, train_pred)))

RMSE on training data:  0.009089491738139825


In [37]:
# Lasso Regression Model
model2 = Lasso(alpha=1.0)
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# use trained Lasso regression model to predict interest rates of training and test data
model2.fit(X, y)
train_pred2=model2.predict(X)
# evaluate model
Lasso_test_pred = cross_val_score(model2, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
# Lasso_test_pred = absolute(scores)
print('RMSE on training data: ', np.sqrt(mean_squared_error(y, train_pred2)))

RMSE on training data:  0.042804183797462224


In [39]:
# XGBoost Regression
import xgboost as xgb
#  Build model 3 using xgb regressor
model3=xgb.XGBRegressor()
# Fit model with data set
model3.fit(X,y)
# Prediction
train_pred3=model3.predict(X)
# Predicted on Test dataset
XGBR_test_pred = model3.predict(df_test)
# Prediction score
# XGBR_test_pred = absolute(scores)
print('RMSE on training data: ', np.sqrt(mean_squared_error(y, train_pred3)))

RMSE on training data:  0.0067954824034316035


In [40]:
# write out csv for each model's test predictions
XGBR_test_pred = pd.DataFrame(XGBR_test_pred, columns=['XGBRPredictions'])

test_predictions = XGBR_test_pred

test_predictions.to_csv("Results from cctsai5 2.csv")

# Discussion on Model Selection and result

Based on RMSE from Linear,Lasso,and XGBoost regression, we can observe that XGBoost regression has the lowest RMSE score,which performs the best and Lasso Regression has the highest score, which performs the worst.Therefore, I pick XGBoost Regression as the final model selection and print out the prediction score on each column.